In [15]:
#! pip install llama-index-llms-openai-like

In [16]:
from dotenv import load_dotenv
import os

load_dotenv()
solar_api_key = os.getenv("SOLAR_API_KEY")

In [17]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
)
from llama_index.vector_stores.milvus import MilvusVectorStore

In [18]:
documents = SimpleDirectoryReader("./city_data/").load_data()

In [19]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [26]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [21]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))

768


In [23]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai_like import OpenAILike

solar_llm = OpenAILike(api_base="https://api.upstage.ai/v1/solar", api_key=solar_api_key, model="solar-1-mini-chat", is_chat_model=True)
response = solar_llm.chat(messages=[
  ChatMessage(role="system", content="You are a helpful assistant."),
  ChatMessage(role="user", content="Hi, how are you?")
])

print(response)

assistant: Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or problems you might have. How can I assist you today?


In [27]:
vector_store = MilvusVectorStore(host="localhost", port=19530, dim=768, overwrite=False, collection_name="solartest")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=solar_llm, embed_model=embed_model)
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context,
    service_context=service_context
)

/var/folders/w0/tssy0xz54jld1pjjx8pl73lm0000gn/T/ipykernel_85523/3877701381.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=solar_llm, embed_model=embed_model)


In [32]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=2048)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        """You are a chatbot capable of answering questions about the data you have in English or Korean"""
    ),
)

In [33]:
chat_engine.chat("What is the size of Seattle?")

AgentChatResponse(response='Seattle has a total area of 142.5 square miles (369 km2), with 83.9 square miles (217 km2) of it being land and 58.7 square miles (152 km2) of it being water.', sources=[ToolOutput(content='system: You are a chatbot capable of answering questions about the data you have in English or Korean\nContext information is below.\n--------------------\nfile_path: /Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt\n\nSeattle (  see-AT-əl; Lushootseed: dᶻidᶻəlal̕ič) is a seaport city on the West Coast of the United States. It is the seat of King County, Washington. With a 2022 population of 749,256 it is the most populous city in both the state of Washington and the Pacific Northwest region of North America. The Seattle metropolitan area\'s population is 4.02 million, making it the 15th-largest in the United States. Its growth rate of 21.1% between 2010 and 2020 made it one of the country\'s fastest-growing large cities.Seattle is situated on an isthm

In [34]:
chat_engine.chat("What are some important landmarks in Seattle? Answer in Korean.")

AgentChatResponse(response="시애틀에는 다음과 같은 중요한 랜드마크들이 있습니다:\n\n1. 스페이스 니들 (Space Needle): 시애틀의 대표적인 랜드마크로, 전망대에서 멋진 도시와 자연 경관을 감상할 수 있습니다.\n2. 파이크 플레이스 마켓 (Pike Place Market): 유명한 농산물 시장으로, 다양한 상점과 레스토랑이 있어 관광객들에게 인기가 있습니다.\n3. 시애틀 그레이트 휠 (Seattle Great Wheel): 엘리엇 베이 워터프론트에 위치한 대형 관람차로, 멋진 도시와 바다 전망을 즐길 수 있습니다.\n4. 치후리 가든 앤 글라스 (Chihuly Garden and Glass): 유명한 유리 예술가 데일 치후리의 작품을 전시하는 박물관입니다.\n5. 시애틀 아트 뮤지엄 (Seattle Art Museum): 다양한 예술 작품을 전시하는 미술관으로, 현대 미술과 전통 예술을 모두 즐길 수 있습니다.\n6. 팝 컬처 박물관 (Museum of Pop Culture): 대중문화와 관련된 전시물과 유물을 볼 수 있는 박물관입니다.\n7. 시애틀 어린이 박물관 (Children's Museum): 어린이들을 위한 교육적인 전시물과 활동을 제공하는 박물관입니다.\n8. 워싱턴 주립 교도소 박물관 (Washington State Reformatory Museum): 과거의 교도소를 박물관으로 개조한 곳으로, 교도소 생활과 역사에 대해 배울 수 있습니다.\n9. 우드랜드 파크 동물원 (Woodland Park Zoo): 다양한 동물들을 관찰할 수 있는 동물원입니다.\n10. 시애틀 워터프론트 (Seattle Waterfront): 엘리엇 베이 주변에 위치한 해안 지역으로, 산책로, 상점, 레스토랑 등이 있어 관광객들에게 인기가 있습니다.\n\n이 외에도 시애틀에는 많은 랜드마크와 관광지가 있으니, 방문하시면 더욱 다양한 경험을 할 수 있을 것입니다.", sources=[ToolOutput(content='system: Yo

AgentChatResponse(response="시애틀에는 다음과 같은 중요한 랜드마크들이 있습니다:\n\n1. 스페이스 니들 (Space Needle): 시애틀의 대표적인 랜드마크로, 전망대에서 멋진 도시와 자연 경관을 감상할 수 있습니다.\n2. 파이크 플레이스 마켓 (Pike Place Market): 유명한 농산물 시장으로, 다양한 상점과 레스토랑이 있어 관광객들에게 인기가 있습니다.\n3. 시애틀 그레이트 휠 (Seattle Great Wheel): 엘리엇 베이 워터프론트에 위치한 대형 관람차로, 멋진 도시와 바다 전망을 즐길 수 있습니다.\n4. 치후리 가든 앤 글라스 (Chihuly Garden and Glass): 유명한 유리 예술가 데일 치후리의 작품을 전시하는 박물관입니다.\n5. 시애틀 아트 뮤지엄 (Seattle Art Museum): 다양한 예술 작품을 전시하는 미술관으로, 현대 미술과 전통 예술을 모두 즐길 수 있습니다.\n6. 팝 컬처 박물관 (Museum of Pop Culture): 대중문화와 관련된 전시물과 유물을 볼 수 있는 박물관입니다.\n7. 시애틀 어린이 박물관 (Children's Museum): 어린이들을 위한 교육적인 전시물과 활동을 제공하는 박물관입니다.\n8. 워싱턴 주립 교도소 박물관 (Washington State Reformatory Museum): 과거의 교도소를 박물관으로 개조한 곳으로, 교도소 생활과 역사에 대해 배울 수 있습니다.\n9. 우드랜드 파크 동물원 (Woodland Park Zoo): 다양한 동물들을 관찰할 수 있는 동물원입니다.\n10. 시애틀 워터프론트 (Seattle Waterfront): 엘리엇 베이 주변에 위치한 해안 지역으로, 산책로, 상점, 레스토랑 등이 있어 관광객들에게 인기가 있습니다.\n\n이 외에도 시애틀에는 많은 랜드마크와 관광지가 있으니, 방문하시면 더욱 다양한 경험을 할 수 있을 것입니다.", sources=[ToolOutput(content='system: You are a chatbot capable of answering questions about the data you have in English or Korean\nContext information is below.\n--------------------\nfile_path: /Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt\n\n== Geography ==\n\n\n=== Topography ===\n\nSeattle is located between the saltwater Puget Sound (an arm of the Pacific Ocean) to the west and Lake Washington to the east. The city\'s chief harbor, Elliott Bay, is part of Puget Sound, which makes the city an oceanic port. To the west, beyond Puget Sound, are the Kitsap Peninsula and Olympic Mountains on the Olympic Peninsula; to the east, beyond Lake Washington and the Eastside suburbs, are Lake Sammamish and the Cascade Range. Lake Washington\'s waters flow to Puget Sound through the Lake Washington Ship Canal (consisting of two man-made canals, Lake Union, and the Hiram M. Chittenden Locks at Salmon Bay, ending in Shilshole Bay on Puget Sound).The sea, rivers, forests, lakes, and fields surrounding Seattle were once rich enough to support one of the world\'s few sedentary hunter-gatherer societies. The surrounding area lends itself well to sailing, skiing, bicycling, camping, and hiking year-round.The city itself is hilly, though not uniformly so. Like Rome, the city is said to lie on seven hills; the lists vary but typically include Capitol Hill, First Hill, West Seattle, Beacon Hill, Queen Anne, Magnolia, and the former Denny Hill. The Wallingford, Delridge, Mount Baker, Seward Park, Washington Park, Broadmoor, Madrona, Phinney Ridge, Sunset Hill, Blue Ridge, Broadview, Laurelhurst, Hawthorne Hills, Maple Leaf, and Crown Hill neighborhoods are all located on hills as well. Many of the hilliest areas are near the city center, with Capitol Hill, First Hill, and Beacon Hill collectively constituting something of a ridge along an isthmus between Elliott Bay and Lake Washington. The break in the ridge between First Hill and Beacon Hill is man-made, the result of two of the many regrading projects that reshaped the topography of the city center. The topography of the city center was also changed by the construction of a seawall and the artificial Harbor Island (completed 1909) at the mouth of the city\'s industrial Duwamish Waterway, the terminus of the Green River. The highest point within city limits is at High Point in West Seattle, which is roughly located near 35th Ave SW and SW Myrtle St.\nNorth of the city center, Lake Washington Ship Canal connects Puget Sound to Lake Washington. It incorporates four natural bodies of water: Lake Union, Salmon Bay, Portage Bay, and Union Bay.Due to its location in the Pacific Ring of Fire, Seattle is in a major earthquake zone. On February 28, 2001, the magnitude 6.8 Nisqually earthquake did significant architectural damage, especially in the Pioneer Square area (built on reclaimed land, as are the Industrial District and part of the city center), and caused one fatality.\nOther strong quakes occurred on January 26, 1700 (estimated at 9 magnitude), December 14, 1872 (7.3 or 7.4), April 13, 1949 (7.1), and April 29, 1965 (6.5). The 1965 quake caused three deaths in Seattle directly and one more by heart failure. Although the Seattle Fault passes just south of the city center, neither it nor the Cascadia subduction zone has caused an earthquake since the city\'s founding. The Cascadia subduction zone poses the threat of an earthquake of magnitude 9.0 or greater, capable of seriously damaging the city and collapsing many buildings, especially in zones built on fill.According to the United States Census Bureau, the city has a total area of 142.5 square miles (369 km2), 83.9 square miles (217 km2) of which is land and 58.7 square miles (152 km2), water (41.16% of the total area).\n\n\n=== Cityscape ===\n\nfile_path: /Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt\n\nVoters in the Puget Sound region approved an additional tax increase in November 2016 to expand light rail to West Seattle and Ballard as well as Tacoma, Everett, and Issaquah.\n\n\n=== Utilities ===\n\nWater and electric power are municipal services, provided by Seattle Public Utilities and Seattle City Light respectively. Other utility companies serving Seattle include Puget Sound Energy (natural gas, electricity), Seattle Steam Company (steam), Waste Management, Inc and Recology CleanScapes (curbside recycling, composting, and solid waste removal), CenturyLink, Frontier Communications, Wave Broadband, and Comcast (telecommunications and television). About 90% of Seattle\'s electricity is produced using hydropower. Less than 2% of electricity is produced using fossil fuels.\n\n\n== International relations ==\nSeattle has the following sister cities:\n\n\n== See also ==\nList of people from Seattle\nList of television shows set in Seattle\nUSS Seattle—two ships\n\n\n== Notes ==\n\n\n== References ==\n\n\n== Bibliography ==\nJones, Nard (1972). Seattle. New York: Doubleday. ISBN 978-0-385-01875-3.\nMorgan, Murray (1982) [1951]. Skid Road: an Informal Portrait of Seattle (revised and updated, first illustrated ed.). Seattle and London: University of Washington Press. ISBN 978-0-295-95846-0.\nOchsner, Jeffrey Karl, ed. (1998) [1994]. Shaping Seattle Architecture: A Historical Guide to the Architects. Seattle and London: University of Washington Press. ISBN 978-0-295-97366-1.\nSale, Roger (1976). Seattle: Past to Present. Seattle and London: University of Washington Press. ISBN 978-0-295-95615-2.\nSpeidel, William C. (1978). Doc Maynard: The Man Who Invented Seattle. Seattle: Nettle Creek Publishing Company. pp. 196–197, 200. ISBN 978-0-914890-02-7.\nSpeidel, William C. (1967). Sons of the profits; or, There\'s no business like grow business: the Seattle story, 1851–1901. Seattle: Nettle Creek Publishing Company. pp. 196–197, 200. ISBN 978-0-914890-00-3.\n\n\n== Further reading ==\nKlingle, Matthew (2007). Emerald City: An Environmental History of Seattle. New Haven: Yale University Press. ISBN 978-0-300-11641-0.\nMacGibbon, Elma (1904). "Seattle, the city of destiny" (DJVU). Leaves of knowledge. Washington State Library\'s Classics in Washington History collection. Shaw & Borden. OCLC 61326250.\nPierce, J. Kingston (2003). Eccentric Seattle: Pillars and Pariahs Who Made the City Not Such a Boring Place After All. Pullman, Washington: Washington State University Press. ISBN 978-0-87422-269-2.\nSanders, Jeffrey Craig. Seattle and the Roots of Urban Sustainability: Inventing Ecotopia (University of Pittsburgh Press; 2010) 288 pages; the rise of environmental activism\n\n\n== External links ==\n\nOfficial website \nHistorylink.org, history of Seattle and Washington\nSeattle Photographs from the University of Washington Digital Collections\nSeattle Historic Photograph Collection from the Seattle Public Library Archived October 23, 2013, at the Wayback Machine\nSeattle Civil Rights and Labor History Project\nSeattle, a National Park Service Discover Our Shared Heritage Travel Itinerary\n--------------------\n', tool_name='retriever', raw_input={'message': 'What are some important landmarks in Seattle? Answer in Korean.'}, raw_output=ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are a chatbot capable of answering questions about the data you have in English or Korean\nContext information is below.\n--------------------\nfile_path: /Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt\n\n== Geography ==\n\n\n=== Topography ===\n\nSeattle is located between the saltwater Puget Sound (an arm of the Pacific Ocean) to the west and Lake Washington to the east. The city\'s chief harbor, Elliott Bay, is part of Puget Sound, which makes the city an oceanic port. To the west, beyond Puget Sound, are the Kitsap Peninsula and Olympic Mountains on the Olympic Peninsula; to the east, beyond Lake Washington and the Eastside suburbs, are Lake Sammamish and the Cascade Range. Lake Washington\'s waters flow to Puget Sound through the Lake Washington Ship Canal (consisting of two man-made canals, Lake Union, and the Hiram M. Chittenden Locks at Salmon Bay, ending in Shilshole Bay on Puget Sound).The sea, rivers, forests, lakes, and fields surrounding Seattle were once rich enough to support one of the world\'s few sedentary hunter-gatherer societies. The surrounding area lends itself well to sailing, skiing, bicycling, camping, and hiking year-round.The city itself is hilly, though not uniformly so. Like Rome, the city is said to lie on seven hills; the lists vary but typically include Capitol Hill, First Hill, West Seattle, Beacon Hill, Queen Anne, Magnolia, and the former Denny Hill. The Wallingford, Delridge, Mount Baker, Seward Park, Washington Park, Broadmoor, Madrona, Phinney Ridge, Sunset Hill, Blue Ridge, Broadview, Laurelhurst, Hawthorne Hills, Maple Leaf, and Crown Hill neighborhoods are all located on hills as well. Many of the hilliest areas are near the city center, with Capitol Hill, First Hill, and Beacon Hill collectively constituting something of a ridge along an isthmus between Elliott Bay and Lake Washington. The break in the ridge between First Hill and Beacon Hill is man-made, the result of two of the many regrading projects that reshaped the topography of the city center. The topography of the city center was also changed by the construction of a seawall and the artificial Harbor Island (completed 1909) at the mouth of the city\'s industrial Duwamish Waterway, the terminus of the Green River. The highest point within city limits is at High Point in West Seattle, which is roughly located near 35th Ave SW and SW Myrtle St.\nNorth of the city center, Lake Washington Ship Canal connects Puget Sound to Lake Washington. It incorporates four natural bodies of water: Lake Union, Salmon Bay, Portage Bay, and Union Bay.Due to its location in the Pacific Ring of Fire, Seattle is in a major earthquake zone. On February 28, 2001, the magnitude 6.8 Nisqually earthquake did significant architectural damage, especially in the Pioneer Square area (built on reclaimed land, as are the Industrial District and part of the city center), and caused one fatality.\nOther strong quakes occurred on January 26, 1700 (estimated at 9 magnitude), December 14, 1872 (7.3 or 7.4), April 13, 1949 (7.1), and April 29, 1965 (6.5). The 1965 quake caused three deaths in Seattle directly and one more by heart failure. Although the Seattle Fault passes just south of the city center, neither it nor the Cascadia subduction zone has caused an earthquake since the city\'s founding. The Cascadia subduction zone poses the threat of an earthquake of magnitude 9.0 or greater, capable of seriously damaging the city and collapsing many buildings, especially in zones built on fill.According to the United States Census Bureau, the city has a total area of 142.5 square miles (369 km2), 83.9 square miles (217 km2) of which is land and 58.7 square miles (152 km2), water (41.16% of the total area).\n\n\n=== Cityscape ===\n\nfile_path: /Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt\n\nVoters in the Puget Sound region approved an additional tax increase in November 2016 to expand light rail to West Seattle and Ballard as well as Tacoma, Everett, and Issaquah.\n\n\n=== Utilities ===\n\nWater and electric power are municipal services, provided by Seattle Public Utilities and Seattle City Light respectively. Other utility companies serving Seattle include Puget Sound Energy (natural gas, electricity), Seattle Steam Company (steam), Waste Management, Inc and Recology CleanScapes (curbside recycling, composting, and solid waste removal), CenturyLink, Frontier Communications, Wave Broadband, and Comcast (telecommunications and television). About 90% of Seattle\'s electricity is produced using hydropower. Less than 2% of electricity is produced using fossil fuels.\n\n\n== International relations ==\nSeattle has the following sister cities:\n\n\n== See also ==\nList of people from Seattle\nList of television shows set in Seattle\nUSS Seattle—two ships\n\n\n== Notes ==\n\n\n== References ==\n\n\n== Bibliography ==\nJones, Nard (1972). Seattle. New York: Doubleday. ISBN 978-0-385-01875-3.\nMorgan, Murray (1982) [1951]. Skid Road: an Informal Portrait of Seattle (revised and updated, first illustrated ed.). Seattle and London: University of Washington Press. ISBN 978-0-295-95846-0.\nOchsner, Jeffrey Karl, ed. (1998) [1994]. Shaping Seattle Architecture: A Historical Guide to the Architects. Seattle and London: University of Washington Press. ISBN 978-0-295-97366-1.\nSale, Roger (1976). Seattle: Past to Present. Seattle and London: University of Washington Press. ISBN 978-0-295-95615-2.\nSpeidel, William C. (1978). Doc Maynard: The Man Who Invented Seattle. Seattle: Nettle Creek Publishing Company. pp. 196–197, 200. ISBN 978-0-914890-02-7.\nSpeidel, William C. (1967). Sons of the profits; or, There\'s no business like grow business: the Seattle story, 1851–1901. Seattle: Nettle Creek Publishing Company. pp. 196–197, 200. ISBN 978-0-914890-00-3.\n\n\n== Further reading ==\nKlingle, Matthew (2007). Emerald City: An Environmental History of Seattle. New Haven: Yale University Press. ISBN 978-0-300-11641-0.\nMacGibbon, Elma (1904). "Seattle, the city of destiny" (DJVU). Leaves of knowledge. Washington State Library\'s Classics in Washington History collection. Shaw & Borden. OCLC 61326250.\nPierce, J. Kingston (2003). Eccentric Seattle: Pillars and Pariahs Who Made the City Not Such a Boring Place After All. Pullman, Washington: Washington State University Press. ISBN 978-0-87422-269-2.\nSanders, Jeffrey Craig. Seattle and the Roots of Urban Sustainability: Inventing Ecotopia (University of Pittsburgh Press; 2010) 288 pages; the rise of environmental activism\n\n\n== External links ==\n\nOfficial website \nHistorylink.org, history of Seattle and Washington\nSeattle Photographs from the University of Washington Digital Collections\nSeattle Historic Photograph Collection from the Seattle Public Library Archived October 23, 2013, at the Wayback Machine\nSeattle Civil Rights and Labor History Project\nSeattle, a National Park Service Discover Our Shared Heritage Travel Itinerary\n--------------------\n', additional_kwargs={}))], source_nodes=[NodeWithScore(node=TextNode(id_='dd5b0d46-5779-41df-b1cd-dece2e307285', embedding=None, metadata={'file_path': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_name': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_type': 'text/plain', 'file_size': 65702, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eba44892-38e1-492d-9a80-34670027feb3', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_name': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_type': 'text/plain', 'file_size': 65702, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}, hash='0bf306055ae49f6d373887f64dcb8b55e706ee8da7b6053b01afb0f56860b930'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='299d330c-5824-4019-805d-04974caa0863', node_type=<ObjectType.TEXT: '1'>, metadata={'file_path': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_name': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_type': 'text/plain', 'file_size': 65702, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}, hash='66faa2e04280e02c79788279bbe4f40e2c78078f3465f42099107edc61bdb1e5'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3e78b378-faad-483a-bd17-94da101c24a8', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='6d6b773f7b95ead5f46e62cdcffdf9939739a93a721a2873911a540a55d03a20')}, text="== Geography ==\n\n\n=== Topography ===\n\nSeattle is located between the saltwater Puget Sound (an arm of the Pacific Ocean) to the west and Lake Washington to the east. The city's chief harbor, Elliott Bay, is part of Puget Sound, which makes the city an oceanic port. To the west, beyond Puget Sound, are the Kitsap Peninsula and Olympic Mountains on the Olympic Peninsula; to the east, beyond Lake Washington and the Eastside suburbs, are Lake Sammamish and the Cascade Range. Lake Washington's waters flow to Puget Sound through the Lake Washington Ship Canal (consisting of two man-made canals, Lake Union, and the Hiram M. Chittenden Locks at Salmon Bay, ending in Shilshole Bay on Puget Sound).The sea, rivers, forests, lakes, and fields surrounding Seattle were once rich enough to support one of the world's few sedentary hunter-gatherer societies. The surrounding area lends itself well to sailing, skiing, bicycling, camping, and hiking year-round.The city itself is hilly, though not uniformly so. Like Rome, the city is said to lie on seven hills; the lists vary but typically include Capitol Hill, First Hill, West Seattle, Beacon Hill, Queen Anne, Magnolia, and the former Denny Hill. The Wallingford, Delridge, Mount Baker, Seward Park, Washington Park, Broadmoor, Madrona, Phinney Ridge, Sunset Hill, Blue Ridge, Broadview, Laurelhurst, Hawthorne Hills, Maple Leaf, and Crown Hill neighborhoods are all located on hills as well. Many of the hilliest areas are near the city center, with Capitol Hill, First Hill, and Beacon Hill collectively constituting something of a ridge along an isthmus between Elliott Bay and Lake Washington. The break in the ridge between First Hill and Beacon Hill is man-made, the result of two of the many regrading projects that reshaped the topography of the city center. The topography of the city center was also changed by the construction of a seawall and the artificial Harbor Island (completed 1909) at the mouth of the city's industrial Duwamish Waterway, the terminus of the Green River. The highest point within city limits is at High Point in West Seattle, which is roughly located near 35th Ave SW and SW Myrtle St.\nNorth of the city center, Lake Washington Ship Canal connects Puget Sound to Lake Washington. It incorporates four natural bodies of water: Lake Union, Salmon Bay, Portage Bay, and Union Bay.Due to its location in the Pacific Ring of Fire, Seattle is in a major earthquake zone. On February 28, 2001, the magnitude 6.8 Nisqually earthquake did significant architectural damage, especially in the Pioneer Square area (built on reclaimed land, as are the Industrial District and part of the city center), and caused one fatality.\nOther strong quakes occurred on January 26, 1700 (estimated at 9 magnitude), December 14, 1872 (7.3 or 7.4), April 13, 1949 (7.1), and April 29, 1965 (6.5). The 1965 quake caused three deaths in Seattle directly and one more by heart failure. Although the Seattle Fault passes just south of the city center, neither it nor the Cascadia subduction zone has caused an earthquake since the city's founding. The Cascadia subduction zone poses the threat of an earthquake of magnitude 9.0 or greater, capable of seriously damaging the city and collapsing many buildings, especially in zones built on fill.According to the United States Census Bureau, the city has a total area of 142.5 square miles (369 km2), 83.9 square miles (217 km2) of which is land and 58.7 square miles (152 km2), water (41.16% of the total area).\n\n\n=== Cityscape ===", start_char_idx=15761, end_char_idx=19296, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.583594799041748), NodeWithScore(node=TextNode(id_='4a58e614-a4f9-41f7-a87c-70add0aac77e', embedding=None, metadata={'file_path': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_name': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_type': 'text/plain', 'file_size': 65702, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eba44892-38e1-492d-9a80-34670027feb3', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_name': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_type': 'text/plain', 'file_size': 65702, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}, hash='0bf306055ae49f6d373887f64dcb8b55e706ee8da7b6053b01afb0f56860b930'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='829a7b86-d448-481c-8da9-77ad18e0cafd', node_type=<ObjectType.TEXT: '1'>, metadata={'file_path': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_name': '/Users/yujiantang/Documents/workspace/rag_cookbooks/data/Seattle.txt', 'file_type': 'text/plain', 'file_size': 65702, 'creation_date': '2024-03-04', 'last_modified_date': '2024-03-04'}, hash='22b4dc53299ceeae402ed8b10d95ef2cab0224fadb608770cd426d388e6f0ec0'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c1d1cea3-2f7e-448a-bb64-a60771f56fdb', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2c67df20384125a0b8eee7f06871ee4cb8d51408c2a3af372ef6f15d57cee078')}, text='Voters in the Puget Sound region approved an additional tax increase in November 2016 to expand light rail to West Seattle and Ballard as well as Tacoma, Everett, and Issaquah.\n\n\n=== Utilities ===\n\nWater and electric power are municipal services, provided by Seattle Public Utilities and Seattle City Light respectively. Other utility companies serving Seattle include Puget Sound Energy (natural gas, electricity), Seattle Steam Company (steam), Waste Management, Inc and Recology CleanScapes (curbside recycling, composting, and solid waste removal), CenturyLink, Frontier Communications, Wave Broadband, and Comcast (telecommunications and television). About 90% of Seattle\'s electricity is produced using hydropower. Less than 2% of electricity is produced using fossil fuels.\n\n\n== International relations ==\nSeattle has the following sister cities:\n\n\n== See also ==\nList of people from Seattle\nList of television shows set in Seattle\nUSS Seattle—two ships\n\n\n== Notes ==\n\n\n== References ==\n\n\n== Bibliography ==\nJones, Nard (1972). Seattle. New York: Doubleday. ISBN 978-0-385-01875-3.\nMorgan, Murray (1982) [1951]. Skid Road: an Informal Portrait of Seattle (revised and updated, first illustrated ed.). Seattle and London: University of Washington Press. ISBN 978-0-295-95846-0.\nOchsner, Jeffrey Karl, ed. (1998) [1994]. Shaping Seattle Architecture: A Historical Guide to the Architects. Seattle and London: University of Washington Press. ISBN 978-0-295-97366-1.\nSale, Roger (1976). Seattle: Past to Present. Seattle and London: University of Washington Press. ISBN 978-0-295-95615-2.\nSpeidel, William C. (1978). Doc Maynard: The Man Who Invented Seattle. Seattle: Nettle Creek Publishing Company. pp. 196–197, 200. ISBN 978-0-914890-02-7.\nSpeidel, William C. (1967). Sons of the profits; or, There\'s no business like grow business: the Seattle story, 1851–1901. Seattle: Nettle Creek Publishing Company. pp. 196–197, 200. ISBN 978-0-914890-00-3.\n\n\n== Further reading ==\nKlingle, Matthew (2007). Emerald City: An Environmental History of Seattle. New Haven: Yale University Press. ISBN 978-0-300-11641-0.\nMacGibbon, Elma (1904). "Seattle, the city of destiny" (DJVU). Leaves of knowledge. Washington State Library\'s Classics in Washington History collection. Shaw & Borden. OCLC 61326250.\nPierce, J. Kingston (2003). Eccentric Seattle: Pillars and Pariahs Who Made the City Not Such a Boring Place After All. Pullman, Washington: Washington State University Press. ISBN 978-0-87422-269-2.\nSanders, Jeffrey Craig. Seattle and the Roots of Urban Sustainability: Inventing Ecotopia (University of Pittsburgh Press; 2010) 288 pages; the rise of environmental activism\n\n\n== External links ==\n\nOfficial website \nHistorylink.org, history of Seattle and Washington\nSeattle Photographs from the University of Washington Digital Collections\nSeattle Historic Photograph Collection from the Seattle Public Library Archived October 23, 2013, at the Wayback Machine\nSeattle Civil Rights and Labor History Project\nSeattle, a National Park Service Discover Our Shared Heritage Travel Itinerary', start_char_idx=62538, end_char_idx=65622, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.5452436208724976)])